## Mining User Data to Create an Index Score 

The factor analysis behind creating an index score can be quite subjective and complicated. One method of generating insights towards factor analysis is to collect and describe data that can help one to analyze connections between factors and outcomes. 

In this report, I analyze factors in a bank's customer data [practice dataset] towards helping to inform the development of a credit scoring system. I seek to analyze a customer's behavior and several customer factors (such as their martical status and family size) and how these might influence their ability to repay a loan. This report answers questions such as: 

- Is there a relation between having kids and repaying a loan on time?
- Is there a relation between marital status and repaying a loan on time?
- Is there a relation between income level and repaying a loan on time?
- How do different loan purposes affect on-time repayment of the loan?

At the conclusion of our analysis, this report is able to recommend that the bank further explore making changes to their credit system based on several insights and predicted impact of customer factors in their ability to repay a loan. This may include tweaking their offers on loan amount and interest amounts based off of the following client information:  

- Family Size 
- Marital Status 
- Income 
- Loan Purpose

This report showed that the proportion of clients who had defaulted on a loan payment was different than the proportion of clients who had not defaulted on a loan, when disaggregated by family size, marital status, income and loan purpose. The difference was typically around 2% for certain factors; for example, we saw that clients who had 0 children, were a widow/widower and took out loans for real estate were less likely to default on a payment. Not surprisingly, we did observe a linear relationship between debt history and income. Our data analysis showed that clients who earned more, were less likely to have defaulted on a loan payment. It is important to dive deeper into this data, and we recommend that the bank define high-income earners and low-income earners, for example. This will help data analysts understand how to weigh the risks and offers to clients in the future. 

### Inspecting Data 

In [1]:
# Importing pandas library
import pandas as pd
# Connecting data source 
data = pd.read_csv('credit_scoring_eng.csv')

In [2]:
#Inspecting data
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### Conclusion

The data has quite a bit of information regarding the banks clients that potentially speak to their ability to repay a loan in a timely manner. Before trying to build a credit scoring system, the data needs to be cleaned as there are several pre-processing issues.

### Data preprocessing

### Processing missing values

When we encounter new datasets and data sources, as is the case here, it is important to sift through the data and look for missing values. Some missing values may be missing at random (MAR), missing completely at random (MCAR) or missing not at random (MNAR). 

We'll need to identify where there are missing values and look for any patterns that we might need to address. Otherwise, we'll have to fill the missing values, most importantly for columns where we will be applying data analysis methods. 

In order to check for missing values such as NaN and None values, we can employ two different methods. One method uses value_counts() and the other uses isnull() to check for missing values. We could use both or one. We'll start by using the value_counts() method and isnull() later on. By using value_count(), we'll also get a head start on identifying duplicate values and noticing patterns in the data. 

In [3]:
# Let's begin by using the value_counts() method and run every column
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In analyzing the first column, we see there is a '-1' listed as a possibility for the number of children. For today's analysis, we will assume that the negative sign is a mistake and will replace it with '1.' We can feel comfortable in making this decision due to the unlikelihood that a client meant to share they had negative one child at the time of providing information the bank. The other error we see is that the data says there are 76 clients with 20 children - this is also most likely an error due to an extra zero. We will go ahead and delete the negative sign and 0 for today's data analysis. 

In working further with the bank, we may seek to find out why these errors appeared and how to ensure they don't occur again.

In [4]:
# Replacing -1 with 1 and replacing the 20 with 2
data['children'] = data['children'].replace({-1:1, 20:2})

In [5]:
print(data['days_employed'].value_counts())

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64


We see more negatives here, in such volume that it becomes clear they are extra symbols that should be deleted. 

In [6]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Here we notice 101 entries for clients of the age "0". Are these different types of accounts that fall under another person's account? Or are these missing values that were input as 0? It is difficult to say what we should do with these values unless we speak to the bank directly. Since we won't be using this column for our data analysis, we'll leave it untouched for now. 

In [7]:
print(data['education'].value_counts())

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64


Data will often be joined with other data, worked on by different people and during this process we can observe discrepancies in spelling. Here we notice that lower() might come in handy to make the data match. 

In [8]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


In [9]:
print(data['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


In [10]:
print(data['family_status'].value_counts())

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


In [11]:
print(data['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [12]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [13]:
print(data['income_type'].value_counts())

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64


In [14]:
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


In [15]:
print(data['total_income'].value_counts())

17312.717    2
31791.384    2
42413.096    2
54857.666    1
26935.722    1
            ..
48796.341    1
34774.610    1
15710.698    1
19232.334    1
9591.824     1
Name: total_income, Length: 19348, dtype: int64


In [16]:
print(data['purpose'].value_counts())

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
housing transactions                        653
buying property for renting out             653
transactions with commercial real estate    651
housing                                     647
purchase of the house                       647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
purchase of my own house                    620
building a property                         620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

Here we see that several purposes for taking out a loan that could be matched, such as "car" and "cars". 

In [17]:
# Let's use isnull() as the next step in our data preprocessing to look for missing values
print(data.columns[data.isnull().any()])

Index(['days_employed', 'total_income'], dtype='object')


### Conclusion

After looking more closely, we see how interesting this bank data is. It spans details about a client's income and lifestyle. Although there was some missing values here and there, and there is still some lingering questions we would need to ask the bank to clean up the data, this dataset looks like a great place to start finding patterns. Hopefully these patterns can help us build a credit scoring system and predict what types of clients are more likely to be good candidates for loans and other bank services. 

### Data type replacement

By using isnull() in the previous step, we find that client information for the number of days they've been employed and their income size is missing for some records. We'll want to replace or fill these missing values depending on how we conduct later data analysis and we should also seek to find out if there is a connection between the missing values. 

*Income Size* 
Income information is listed as a quantitative variable. Missing values in quantitative variables are filled with representative values. These values represent the status of the data set selected for analysis. To estimate the typical income size and replace the missing values in this column, we'll use the average income size (we might also look to use the median in another case).

In [18]:
# Replacing missing values in the column showing income size of a client 
# Creating a representative value first and saving it to total_income_avg
total_income_avg = data['total_income'].mean()
data['total_income'] = data['total_income'].fillna(value = total_income_avg)

*Employment History* 
When it comes to employment history, we can also fill the missing values with the average number of days employed for the group as a whole. This is a temporary fix, as we won't actually be using this data for our analysis. This is because further exploration reveals that the days_employed column has many faults, not just the additional negative signs, the missing values, but also impossible data. When we print this column to show the highest values first, we find that some clients have around 40,000 days employed, which comes to out around 1096 years. This is quite impossible! 

Let's take a look below and see for ourselves. We'll also go ahead and make a couple of corrections, like changing the data type from float to integer and filling the missing rows with the average number of days employed. 

(With correct data, we would first delete the negatives and then fill missing values with the median of the column. The median is less sensitive to outliers). 

In [19]:
# Sorting values for 'days_employed' to show the highest values 
print(data.sort_values(by='days_employed', ascending=False)['days_employed'].head(5))

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
Name: days_employed, dtype: float64


In [20]:
# Changing data type from float to integer
data['days_employed'] = round(data['days_employed'])

In [21]:
# Replacing missing values in the column showing the number of days a client has been employed
# Creating a representative value first and saving it to days_employed_avg
days_employed_avg = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(value = days_employed_avg)

### Conclusion

After seeing the impossible values in the days_employed column, we can brainstorm reasons why these values are so large. One hypothesis is that the decimal is placed incorrectly. In order to confirm this hypothesis, we would need to consult with the bank and make the correct changes. In the meantime, we're satisfied with having changed the data type and filling in the missing values. 

Now that we have replaced missing values in the data for income size, we can feel confident in performing future calculations that integrate this information. Now, before we move on to identifying patterns in the data, we should make sure that we don't have any duplicate data. 

### Processing duplicates

The next step in our data preprocessing will be to delete duplicate values. We might be curious to see what, if any, columns have duplicates. To spot duplicates, we will used duplicated() to highlight the rows that have duplicate values across most of the columns in our dataset. This would indicate that the higlighted row has a duplicate and should be removed. We will then follow up by using drop_duplicates(), followed by reset_index() to avoid creating a column with old index values. 

In [22]:
# Counting number of rows before dropping duplicates
print(len(data))

21525


In [23]:
# Spot duplicates 
duplicate_data = data.duplicated(subset = ['children', 'days_employed', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'debt', 'total_income', 'purpose'], keep = 'first') 
print(duplicate_data.value_counts())

False    21454
True        71
dtype: int64


In [24]:
# Delete duplicate rows 
data = data.drop_duplicates(['children', 'days_employed', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'debt', 'total_income', 'purpose']).reset_index(drop = True) 

In [25]:
# Counting number of rows after dropping duplicates
print(len(data))

21454


### Conclusion

Perfect! We now have data rid of duplicate values. 71 duplicates to be exact. We decided to use drop_duplicates as an easy way to spot the 71 duplicate rows. We used multiple columns (most) to filter and ensure that we were really deleting duplicate entries. These 71 rows are most likely entries that were made twice at the time a client opened up an account at this bank. There are far too many similarities in all of the columns to indicate any other hypothesis for why the duplicate rows were in this dataseet. 

### Categorizing Data

Now that our data has been prepared for analysis, we can really get started on finding patterns and trying to make insights. The data has information on client's loan history and if they were able to repay the loan on time. Since the bank would like to generally understand borrower's risk of defaulting, we will definitely want to break out our analysis by loan type. Do clients who take out loans for weddings pay their loan back more often than those who take out loans for real estate purposes? 

To answer this question and several others, we'll start by grouping our loans by type or category. In the code below, we will use replace() to group similar loan types all under once identifier. For example, both 'buying my own car' and 'second-hand car purchase' are now both identified by the string, 'car'.

In [26]:
# Creating a new column 'purpose_grouped' to house the categories for grouped loan types 
data['purpose_grouped'] = data['purpose']

In [27]:
# Grouping loan types under the categories 'wedding', 'real estate', 'car' and 'education'
data['purpose_grouped'] = data['purpose_grouped'].replace({'wedding ceremony':'wedding', 'having a wedding':'wedding', 'to have a wedding':'wedding'})
data['purpose_grouped'] = data['purpose_grouped'].replace({'real estate transactions':'real estate', 'buy commercial real estate':'real estate', 'housing transactions':'real estate', 'buying property for renting out':'real estate', 'housing transactions':'real estate', 'transactions with commercial real estate':'real estate', 'purchase of the house':'real estate', 'housing':'real estate', 'purchase of the house for my family':'real estate', 'construction of own property':'real estate','property':'real estate','transactions with my real estate':'real estate','building a real estate':'real estate','buy real estate':'real estate','purchase of my own house':'real estate','building a property':'real estate','housing renovation':'real estate','buy residential real estate':'real estate'})
data['purpose_grouped'] = data['purpose_grouped'].replace({'buying my own car':'car','car':'car','second-hand car purchase':'car','to own a car':'car','respon':'car','buying a second-hand car':'car','cars':'car','to buy a car':'car','car purchase':'car','purchase of a car':'car'})
data['purpose_grouped'] = data['purpose_grouped'].replace({'going to university':'education', 'supplementary education':'education', 'university education':'education', 'to get a supplementary education':'education', 'getting an education':'education', 'profile education':'education', 'getting higher education':'education', 'to become educated':'education'})
# Print(data.head(5))
print(data['purpose_grouped'].value_counts())

real estate    10811
car             4306
education       4013
wedding         2324
Name: purpose_grouped, dtype: int64


### Conclusion

Great! All of our loan types are now in neat categories that will help us in the next steps. The variations in how the loan types were written (i.e., car and cars) may have arisen from different contributions made by information architects or database managers. Over the years, the identifiers for loan types may have changed. 

We also see that loans due to real estate purposes are by far the most frequent type of loan managed by this bank. 

### Data Analysis

#### - Is there a relation between having kids and repaying a loan on time?

To answer this question we'll want to use a pivot table. Pivot tables work well when it comes to presenting and synthesizing data.

In [28]:
import numpy as np

In [29]:
# Converting children data type to object 
data['children'] = data['children'].astype('object')

In [30]:
# Creating a pivot table
pivot_children = pd.pivot_table(data, index = ['children'], 
                                values = ['debt'], aggfunc = [np.sum, len])

In [31]:
# Here I am iterating the data columns names for my own understand of syntax
for col in pivot_children.columns: 
    print(col)

('sum', 'debt')
('len', 'debt')


In [32]:
# Adding a column showing the proportion of clients who have defaulted on a loan payment
pivot_children['proportion'] = (pivot_children[('sum','debt')]/pivot_children[('len', 'debt')])*100
pivot_children.sort_values(by='proportion', ascending = False)

,sum,len,proportion
,debt,debt,
children,,,
4,4,41,9.756098
2,202,2128,9.492481
1,445,4855,9.165808
3,27,330,8.181818
0,1063,14091,7.543822
5,0,9,0.000000


### Conclusion

The pivot table above shows that of clients
- reporting 4 children, only 9.76% have ever defaulted on a loan payment. 
- reporting 2 children, only 9.49% have ever defaulted on a loan payment. 
- reporting 1 children, only 9.17% have ever defaulted on a loan payment. 
- reporting 3 children, only 8.18% have ever defaulted on a loan payment. 
- reporting 0 children, only 7.54% have ever defaulted on a loan payment. 

It is not recommended to draw conclusions regarding clients reporting 5 children, because the size is small (n = 9) and we cannot speak as to it's validity as a representative sample. The other group sizes are large enough where we could conduct further data analysis. 

The variations between proportions of clients with debt and no debt history when analyzed by family size are are not linear (I.e., the larger the family size, the more debt). In a lengthier data analysis, we may seek to run a t-test to find the correlation and strength of the correlation between the proportions. Through these statistical tests, we may find that there is a statistically significant difference between specific family sizes. This type of statistical test is not always appropriate or can be deemed unecessary, and the bank may not be seeking this type of analysis. 

For today's analysis, we  can conclude that there is no linear relationship between family size and debt history. We do howerver, observe small differences in between groups. For example, clients reporting 0 children are the least likely to default on a loan payment. 

#### - Is there a relation between marital status and repaying a loan on time?

In [33]:
# Creating a pivot table
pivot_marital = pd.pivot_table(data, index = ['family_status'], 
                               values = ['debt'], aggfunc = [np.sum, len])

In [34]:
# Adding a column showing the proportion of clients who have defaulted on a loan payment
pivot_marital['proportion'] = (pivot_marital[('sum','debt')]/pivot_marital[('len', 'debt')])*100
print(pivot_marital.sort_values(by='proportion', ascending = False))

                   sum    len proportion
                  debt   debt           
family_status                           
unmarried          274   2810   9.750890
civil partnership  388   4151   9.347145
married            931  12339   7.545182
divorced            85   1195   7.112971
widow / widower     63    959   6.569343


### Conclusion

The pivot table above shows that of clients who are
- in a civil partnership, only 9.35% have ever defaulted on a loan payment. 
- divorced, only 7.11% have ever defaulted on a loan payment. 
- married, only 7.55% have ever defaulted on a loan payment. 
- unmarried, only 9.75% have ever defaulted on a loan payment. 
- a widow/widower, only 6.57% have ever defaulted on a loan payment. 

There is hint of a potentially significant difference for clients reporting being a widow or widower and their loan repayment history. At 6.56% debt history, this group is the least likely to miss a loan payment. It also appears that unmarried clients have a bit higher rates of defaulting on a debt, coming in at 9.75%. 

#### - Is there a relation between income level and repaying a loan on time?

In order to answer this question, we should categorize this data to show groups of clients who are "high income" and those who are "low income". This categorization will help us to see patterns in the data in this analysis.


In [35]:
# We are first going to look at the minimum and maximum values for income 
# This will help us to know how we can build our function
# In a more detailed data analysis, we may want to build a scatterplot or histogram as well
print(data['total_income'].max())
print(data['total_income'].min())

362496.645
3306.762


In [36]:
# Creating a function to group incomes by type 
def income_function(income):
    if income >= 100000:
        return 'high'
    if income >= 50000:
        return 'medium'
    if income >= 0:
        return 'low'

In [37]:
# Creating a new column called total_income_grouped 
# Applying the income_function to the total_income column 
data['total_income_grouped'] = data['total_income'].apply(income_function)

In [38]:
# Looking at how many clients in this dataset fall into low, medium high income 
print(data['total_income_grouped'].value_counts())

low       20134
medium     1221
high         99
Name: total_income_grouped, dtype: int64


In [39]:
# Creating a pivot table
pivot_income = pd.pivot_table(data, index = ['total_income_grouped'], 
                               values = ['debt'], aggfunc = [np.sum, len])

In [40]:
# Adding a column showing the proportion of clients who have defaulted on a loan payment
pivot_income['proportion'] = (pivot_income[('sum','debt')]/pivot_income[('len', 'debt')])*100
print(pivot_income.sort_values(by='proportion', ascending = False))

                       sum    len proportion
                      debt   debt           
total_income_grouped                        
low                   1649  20134   8.190126
medium                  86   1221   7.043407
high                     6     99   6.060606


### Conclusion

The pivot table shows us a linear relationship between income size and debt history. This indicates that potentially, clients with higher incomes are less likely to default on a loan payment. 

In a lengthier data analysis and with more information from the bank, we would also want to test the strength of the correlation. In speaking with the bank further, we would want to clarify how they would like to define the different income groups (I.e., How much money does a high-income earner bring in?) 

#### - How do different loan purposes affect on-time repayment of the loan?

In [41]:
# Creating a pivot table 
pivot_purpose = pd.pivot_table(data, index = ['purpose_grouped'], 
                               values = ['debt'], aggfunc = [np.sum, len])

In [42]:
#adding a column showing the proportion of clients who have defaulted on a loan payment
pivot_purpose['proportion'] = (pivot_purpose[('sum','debt')]/pivot_purpose[('len', 'debt')])*100
print(pivot_purpose.sort_values(by='proportion', ascending = False))

                 sum    len proportion
                debt   debt           
purpose_grouped                       
car              403   4306   9.359034
education        370   4013   9.220035
wedding          186   2324   8.003442
real estate      782  10811   7.233373


### Conclusion

The pivot table above shows that of clients who are seeking a loan related to 
- cars, only 9.36% have ever defaulted on a loan payment. 
- education, only 9.22% have ever defaulted on a loan payment. 
- real estate, only 7.23% have ever defaulted on a loan payment. 
- wedding, only 8.00% have ever defaulted on a loan payment. 

There is hint of a potentially significant difference for clients reporting taking out a loan for real estate and their loan repayment history. At 7.23% debt history, this group is the least likely to miss a loan payment. It also appears that clients taking out car loans have a bit higher rates of defaulting on a debt, coming in at 9.36%. 

### General conclusion

Overall, this report recommends that the bank has reason enough to further explore making changes to their credit system. This may include tweaking their offers on loan amount and interest amounts based off of the following client information:  

- Family Size 
- Marital Status 
- Income 
- Loan Purpose

We recommend that the bank create an index score that gives individual scores given to clients based off of their marital status, family size, income, and loan purpose. When aggregated, the index can be representative of the likelihood that a client will never miss a loan payment.

This report showed that the proportion of clients who had defaulted on a loan payment was different than the proportion of clients who had not defaulted on a loan, when disaggregated by family size, marital status, income and loan purpose. 

The difference was typically around 2% for certain factors; for example, we saw that clients who had 0 children, were a widow/widower and took out loans for real estate were less likely to default on a payment. 

Not surprisingly, we did observe a linear relationship between debt history and income. Our data analysis showed that clients who earned more, were less likely to have defaulted on a loan payment. It is important to dive deeper into this data, and we recommend that the bank define high-income earners and low-income earners, for example. This will help data analysts understand how to weigh the risks and offers to clients in the future. 

The next step of data analysis, should include further discussion with the bank regarding their goals and data. The bank may seek to conduct further testing of stastitically significant differences between groups and build a credit index with varying degrees of weight on each factor. 